In [370]:
# we have six functions:
# calcEntropy:  calculate the entropy
# calcSexDisc ：#Calculate Disc(D)
# majorityCnt:  Based on the most occur feature to choice
# splitDataSet:  Based on the best choice, split the dataset.
# chooseBestFeatureSplit:  Based on the FIG choice the best position.
# createTree:  Build the decision tree.


In [371]:
import numpy as np
#dataset
#KDD Census-Income target(binary >50k or <=50k)
import pandas as pd
dt = pd.read_csv('adult.csv')
features = ["age","workclass","fnlwgt","education","education.num","marital.status","occupation","relationship","race","sex","capital.gain","capital.loss","hours.per.week","native.country","income"]
da = dt.replace('?',np.nan,inplace=False)
dataset = da.dropna(axis=0,how='any',thresh = None,subset= None,inplace=False)
X = dataset.iloc[:, :14].values
dataset

age workclass  fnlwgt     education  education.num      marital.status  \
1       82   Private  132870       HS-grad              9             Widowed   
3       54   Private  140359       7th-8th              4            Divorced   
4       41   Private  264663  Some-college             10           Separated   
5       34   Private  216864       HS-grad              9            Divorced   
6       38   Private  150601          10th              6           Separated   
...    ...       ...     ...           ...            ...                 ...   
32556   22   Private  310152  Some-college             10       Never-married   
32557   27   Private  257302    Assoc-acdm             12  Married-civ-spouse   
32558   40   Private  154374       HS-grad              9  Married-civ-spouse   
32559   58   Private  151910       HS-grad              9             Widowed   
32560   22   Private  201490       HS-grad              9       Never-married   

              occupation   relationship   race     sex  capital.gain  \
1        Exec-managerial  Not-in-family  White  Female             0   
3      Machine-op-inspct      Unmarried  White  Female             0   
4         Prof-specialty      Own-child  White  Female             0   
5          Other-service      Unmarried  White  Female             0   
6           Adm-clerical      Unmarried  White    Male             0   
...                  ...            ...    ...     ...           ...   
32556    Protective-serv  Not-in-family  White    Male             0   
32557       Tech-support           Wife  White  Female             0   
32558  Machine-op-inspct        Husband  White    Male             0   
32559       Adm-clerical      Unmarried  White  Female             0   
32560       Adm-clerical      Own-child  White    Male             0   

       capital.loss  hours.per.week native.country income  
1              4356              18  United-States  <=50K  
3              3900              40  United-States  <=50K  
4              3900              40  United-States  <=50K  
5              3770              45  United-States  <=50K  
6              3770              40  United-States  <=50K  
...             ...             ...            ...    ...  
32556             0              40  United-States  <=50K  
32557             0              38  United-States  <=50K  
32558             0              40  United-States   >50K  
32559             0              40  United-States  <=50K  
32560             0              20  United-States  <=50K  

[30162 rows x 15 columns]

In [372]:
from math import log

def calcEntropy(date):
	num_entries = len(date)
	label_counts = {}								#Saves a dictionary of the number of occurrences of each label
	for featVec in date:
		currentLabel = featVec[-1]					#get Label information
		if currentLabel not in label_counts.keys():
			label_counts[currentLabel] = 0
		label_counts[currentLabel] += 1				#label quantity
	calc_ent = 0.0									#entropy
	for key in label_counts:
		prob = float(label_counts[key]) / num_entries	#calculate probability
		calc_ent -= prob * log(prob, 2)					#Calculate entropy
		return calc_ent


In [373]:
def calcSexDisc(data):								#Calculate Disc(D)
	num_entries = len(data)
	male_greater50 = 0
	female_greater50 = 0
	male_count = 0
	female_count = 0
	for featVec in data:
		for item in featVec:
			if item == 'Male':
				male_count += 1
				if item == 'Male' and featVec[-1] =='>50K':
					male_greater50 += 1
			elif item == 'Female':
				female_count += 1
				if item == 'Female' and featVec[-1] == '>50K':
					female_greater50 += 1
	if male_count == 0 or female_count == 0 :
		return 0
	disc_female = (male_greater50 / male_count) / (male_count / num_entries) - (female_greater50 / female_count) / (female_count / num_entries)
	return disc_female


In [374]:
import operator

def majorityCnt(classList):
	classCount = {}
	for vote in classList:							#Counts the number of occurrences of each element in the classList
		if vote not in classCount.keys():
			classCount[vote] = 0
		classCount[vote] += 1
	#Sort in descending order according to the dict value
	sorted_class_count = sorted(classCount.items(), key = operator.itemgetter(1), reverse = True)
	return sorted_class_count[0][0]					#Returns the most frequent occurrence of the element in the classList


In [375]:
def chooseBestFeatureSplit(dataset):
	num_features = len(dataset[0]) - 1					#Feature quantity
	base_entropy = calcEntropy(dataset) 				#Calculate entropy when you're not doing anything
	base_disc = calcSexDisc(dataset)
	best_info_gain = 0.0
	best_feature = -1									#index of the optimal feature
	best_fig = 0.0
	for i in range(num_features): 						#traverse all feature
		feat_list = [example[i] for example in dataset]
		unique_vals = set(feat_list)     					   #Create a set {} with non-repeatable elements
		new_entropy = 0.0
		new_disc = 0.0
		for value in unique_vals:
			sub_data_set = splitDataSet(dataset, i, value) 		#dataset after split
			prob = len(sub_data_set) / float(len(dataset))
			new_entropy += prob * calcEntropy(sub_data_set) 	#Calculate entropy for each feature
			new_disc += prob * abs(calcSexDisc(sub_data_set))
		info_gain = base_entropy - new_entropy 					#information gain
		fair_gain = abs(base_disc) - new_disc
		#print("第%d个特征的增益为%.3f" % (i, info_gain))			#info gain per feature
		if info_gain > best_info_gain:
			best_info_gain = info_gain 							#Update the info gain to find the maximum information gain
			best_feature = i
		if fair_gain != 0:
			fi_gain = fair_gain * best_info_gain
		else:
			fi_gain = best_info_gain
		if fi_gain > best_fig:
			best_fig = fi_gain
			best_feature = i
	return best_feature

In [376]:
def createTree(dataset, labels, featLabels):
	class_list = [example[-1] for example in dataset]
	if class_list.count(class_list[0]) == len(class_list):		#If the categories are exactly the same, the division stops
		return class_list[0]
	if len(dataset[0]) == 1 or len(labels) == 0:			    #Returns the class label that appears most often
		return majorityCnt(class_list)
	best_feat = chooseBestFeatureSplit(dataset)				    #Select the best feature
	best_feat_label = labels[best_feat]							#Select the best label
	featLabels.append(best_feat_label)							#root
	my_tree = {best_feat_label:{}}
	del(labels[best_feat])										#remove used feature labels
	feat_values = [example[best_feat] for example in dataset]	#Get the labels values of all the best features in the training set
	unique_vals = set(feat_values)								#Remove duplicate labels values
	for value in unique_vals:
		sub_labels = labels[:]
		my_tree[best_feat_label][value] = createTree(splitDataSet(dataset, best_feat, value), sub_labels, featLabels)  #recursion
	return my_tree

In [377]:
def splitDataSet(dataset, target, value):
	ret_data_set = []
	for feat_vec in dataset:
		if feat_vec[target] == value:
			reduced_feat_vec = feat_vec[:target]				#remove target feature
			reduced_feat_vec.extend(feat_vec[target+1:])
			ret_data_set.append(reduced_feat_vec)
	return ret_data_set


In [378]:

X_train = X[1000:]
X_test = X[:1000]
featLabels=[]
decisiontree = createTree(X_train.tolist(), features,featLabels)

